In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
all_file_path = glob('dataverse_files/*.edf')
all_file_path

['dataverse_files/s01.edf',
 'dataverse_files/h01.edf',
 'dataverse_files/h14.edf',
 'dataverse_files/s14.edf',
 'dataverse_files/s02.edf',
 'dataverse_files/h02.edf',
 'dataverse_files/h03.edf',
 'dataverse_files/s03.edf',
 'dataverse_files/s07.edf',
 'dataverse_files/s13.edf',
 'dataverse_files/h13.edf',
 'dataverse_files/h07.edf',
 'dataverse_files/h06.edf',
 'dataverse_files/h12.edf',
 'dataverse_files/s12.edf',
 'dataverse_files/s06.edf',
 'dataverse_files/s10.edf',
 'dataverse_files/s04.edf',
 'dataverse_files/h04.edf',
 'dataverse_files/h10.edf',
 'dataverse_files/h11.edf',
 'dataverse_files/h05.edf',
 'dataverse_files/s05.edf',
 'dataverse_files/s11.edf',
 'dataverse_files/s08.edf',
 'dataverse_files/h08.edf',
 'dataverse_files/h09.edf',
 'dataverse_files/s09.edf']

In [3]:
all_file_path[0]

'dataverse_files/s01.edf'

In [4]:
healthy_file_path = [i for i in all_file_path if 'h' in i.split('/')[1]]
patient_file_path =  [i for i in all_file_path if 's' in i.split('/')[1]]
print(len(patient_file_path))


14


In [5]:
healthy_file_path

['dataverse_files/h01.edf',
 'dataverse_files/h14.edf',
 'dataverse_files/h02.edf',
 'dataverse_files/h03.edf',
 'dataverse_files/h13.edf',
 'dataverse_files/h07.edf',
 'dataverse_files/h06.edf',
 'dataverse_files/h12.edf',
 'dataverse_files/h04.edf',
 'dataverse_files/h10.edf',
 'dataverse_files/h11.edf',
 'dataverse_files/h05.edf',
 'dataverse_files/h08.edf',
 'dataverse_files/h09.edf']

In [6]:
patient_file_path

['dataverse_files/s01.edf',
 'dataverse_files/s14.edf',
 'dataverse_files/s02.edf',
 'dataverse_files/s03.edf',
 'dataverse_files/s07.edf',
 'dataverse_files/s13.edf',
 'dataverse_files/s12.edf',
 'dataverse_files/s06.edf',
 'dataverse_files/s10.edf',
 'dataverse_files/s04.edf',
 'dataverse_files/s05.edf',
 'dataverse_files/s11.edf',
 'dataverse_files/s08.edf',
 'dataverse_files/s09.edf']

In [44]:
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload= True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs = mne.make_fixed_length_epochs(data,duration = 5, overlap = 1)
    array = epochs.get_data()
    return array
    

In [45]:
sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /Users/karandeepsinghnanda/Documents/VSCode/EEG-Classification/dataverse_files/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
231 matching events found
No baseline corre

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


0 bad epochs dropped


In [46]:
sample_data.shape #(No of epochs, channels, length of signals)

(231, 19, 1250)

In [47]:
%%capture
control_epochs_array = [read_data(i) for i in healthy_file_path]
patient_epochs_array = [read_data(i) for i in patient_file_path]

In [48]:
control_epochs_array[0].shape

(231, 19, 1250)

In [49]:
control_epochs_labels = [len(i) * [0] for i in control_epochs_array]
patient_epochs_labels = [len(i) * [1] for i in patient_epochs_array]
len(control_epochs_labels), len(patient_epochs_labels)

(14, 14)

In [50]:
data_list = control_epochs_array + patient_epochs_array
label_list = control_epochs_labels + patient_epochs_labels

In [51]:
group_list = [[i] * len(j) for i,j in enumerate(data_list)]
len(group_list)

28

In [52]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)
print(data_array.shape, label_array.shape, group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


## Feature Extraction


In [53]:
from scipy import stats

def mean(x):
    return np.mean(x, axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x, axis=-1)
def minim(x):
    return np.min(x, axis=-1)
def maxim(x):
    return np.max(x, axis=-1)
def argminim(x):
    return np.argmin(x, axis=-1)
def argmaxim(x):
    return np.argmax(x, axis=-1)

def rms(x): #Root mean square
    return np.sqrt(np.mean(x **2, axis = -1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis =-1)

def skewness(x):
    return stats.skew(x, axis =-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)


def concatenate_features(x):
    return np.concatenate((mean(x),std(x),ptp(x),var(x),minim(x), maxim(x),argminim(x), 
                           argmaxim(x),rms(x),abs_diff_signal(x), skewness(x),kurtosis(x)), axis=-1)


In [54]:
features = []
for d in data_array:
    features.append(concatenate_features(d))

In [55]:
features_array = np.array(features)
features_array.shape


(7201, 228)

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

In [57]:
clf = LogisticRegression()
gkfold = GroupKFold(5)
pipe = Pipeline([('scaler', StandardScaler()), ('clf' , clf)])
param_grid = {'clf__C' : [0.1,0.5,0.7,1,3,5,7]}
gscv = GridSearchCV(pipe, param_grid, cv= gkfold, n_jobs = 12)
gscv.fit(features_array, label_array, groups = group_array)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=12, param_grid={'clf__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [58]:
gscv.best_score_

0.6560122552045833

## Deep Learning model CNN (1D)